## Сбор данных 

**Импорт библиотек:**

In [2]:
import requests
import numpy as np
import pandas as pd
import yaml

**Считываем токен из `.yaml` файла:**

In [3]:
conf = open('config.yaml')

config = yaml.load(conf, yaml.SafeLoader)

access_token = config['access_token']

**Считываем описание признаков из `.yaml` файла:**

In [4]:
descr = open('description/description_search.yaml', mode = 'r', encoding = 'utf-8')

description = yaml.load(descr, yaml.SafeLoader)

**Подготовка к API запросу. Используется метод `users.get`:**

In [24]:
url = 'https://api.vk.com/method/users.get?'

params = {
    'v': 5.131,
    'access_token': access_token,
    'fields': 'relation, personal, universities, sex'
}

**Функция, которая 'раскрывает' словарь:**

In [25]:
def del_dict(dictionary, del_dict):
        
    if not del_dict in dictionary.keys():
        return

    if type(dictionary[del_dict]) != list:
        for key in dictionary[del_dict].keys():
            dictionary[del_dict + '_' + key] = dictionary[del_dict][key]
        
    elif len(dictionary[del_dict]) != 0:
        for key in dictionary[del_dict][0].keys():
            dictionary[del_dict + '_' + key] = dictionary[del_dict][0][key]
            
    dictionary.pop(del_dict)    
    
    return dictionary

**Функция, которая удаляет словарь из основного словаря:**

In [26]:
def del_part(dictionary, del_dict):
    
    if not del_dict in dictionary.keys():
        return
    
    dictionary.pop(del_dict)
    
    return dictionary

**Функция, которя возрващает `DateFrame` путем API запросов:**

In [27]:
def get_data(count=1000):
    
    user_str = ''

    user_id_list = np.random.randint(1, 60000000, count)

    for user_id in user_id_list:
        user_str = user_str + str(user_id) + ', '
    
    df = pd.DataFrame(columns=description.keys())
    
    params['user_ids'] = user_str
        
    r = requests.get(url, params = params)
    
    data_dict = r.json()['response']
    
    for user_dict in data_dict:
        
        if  (user_dict['first_name'] != 'DELETED') and \
            ('deactivated' not in user_dict.keys()) and \
            (user_dict['is_closed'] == False) and \
            (user_dict['sex'] != 0):
        
            del_dict(user_dict, 'personal')
            del_dict(user_dict, 'universities')
            del_part(user_dict, 'relation_partner')
            
            df = df.append(user_dict, ignore_index=True)
            
    return df

**Фунция, которая получает минимальное кол-во пользователей, которых необходимо случайно выбрать и выдет `.csv` файл, состоящий из этих пользователей:**

In [28]:
def get_users_csv(min_count, file_name):
    
    file_name = file_name + '.csv' 
    
    full_df = pd.DataFrame(columns=description.keys())
    
    while len(full_df) < min_count:
    
        df = get_data()
    
        full_df = pd.concat([full_df, df])
        
        full_df.to_csv(file_name, index_label = 'id')
    
    full_df = full_df.drop_duplicates(subset=['id'])
    
    full_df = full_df.set_index('id')
        
    return full_df

**Заполнение паппки `data` данными, в среднем по 25000 пользователей:**

In [35]:
%%time
data_list = [5, 6, 7, 8, 9, 10]

for count in data_list:
    
    get_users_csv(25000, file_name='data/data_'+str(count))

Wall time: 23min 14s
